In [ ]:
import psycopg2
import psycopg2.extras as extras
import pandas as pd
import json
from datetime import datetime
from google.cloud import bigquery
from dotenv import dotenv_values
from dotenv import dotenv_values
import re
#pip uninstall psycopg
#pip install psycopg2

In [ ]:
project_id='pongthorn'
env_path='../.env'
config = dotenv_values(dotenv_path=env_path)
# config

In [ ]:
start_date_query='2023-07-01'
end_date_query='2023-11-30' 

In [ ]:
dt_imported=datetime.now()
str_imported=dt_imported.strftime('%Y-%m-%d %H:%M:%S')
print(f"Imported DateTime: {str_imported}" )


In [ ]:
def get_postgres_conn():
 try:
  conn = psycopg2.connect(
        database=config['DATABASES_NAME'], user=config['DATABASES_USER'],
      password=config['DATABASES_PASSWORD'], host=config['DATABASES_HOST']
     )
  return conn

 except Exception as error:
  print(error)      
  raise error
def list_data(sql,params,connection):
 df=None   
 with connection.cursor() as cursor:
    
    if params is None:
       cursor.execute(sql)
    else:
       cursor.execute(sql,params)
    
    columns = [col[0] for col in cursor.description]
    dataList = [dict(zip(columns, row)) for row in cursor.fetchall()]
    df = pd.DataFrame(data=dataList) 
 return df 

In [ ]:
#incident.id,  order by id

sql_incident=f"""

select
incident.id as id,incident.incident_no as incident_no
,status.incident_status_name as status,severity.severity_name as  severity
,incident.incident_subject as subject,incident.incident_description as description
,TO_CHAR(incident.incident_datetime  AT TIME ZONE 'Asia/Bangkok','YYYY-MM-DD HH24:MI') as open_datetime
,TO_CHAR(incident.incident_close_datetime  AT TIME ZONE 'Asia/Bangkok','YYYY-MM-DD HH24:MI') as close_datetime
,xtype.incident_type_name as incident_type,service.service_type_name service_type
,inventory.serial_number
-- ,service_level.sla_name as sla
,product_type.productype_name as product_type,brand.brand_name as brand,model.model_name as model


from app_incident as incident
inner join app_incident_type as  xtype on incident.incident_type_id = xtype.id
inner join  app_incident_status as status on incident.incident_status_id = status.id
inner join  app_incident_severity as severity on  incident.incident_severity_id = severity.id
inner join  app_service_type as service on incident.service_type_id= service.id
inner join app_inventory as inventory on incident.inventory_id = inventory.id
inner join app_brand as brand on inventory.brand_id = brand.id
inner join app_model as model on inventory.model_id = model.id
inner join app_product_type as product_type on inventory.product_type_id = product_type.id
inner join app_sla as service_level on inventory.customer_sla_id = service_level.id

-- inner join app_project as project on inventory.project_id = project.id
-- inner join app_company as company on project.company_id = company.id

where incident_datetime>='{start_date_query}'
and incident.incident_status_id=4
and inventory.is_dummy=False


"""
#and  incident.incident_datetime<=%(end_date_param)s

In [ ]:

# sql_detail="""

# select  detail.id, detail.incident_master_id as incident_id ,
# workaround_resolution as resolution,
# TO_CHAR(task_start,'YYYY-MM-DD HH24:MI:SS') as task_start,
# TO_CHAR(task_end,'YYYY-MM-DD HH24:MI:SS') as task_end,
# team.service_team_name
# from app_incident_detail detail
# inner join  app_serviceteam team on detail.service_team_id=team.id
# inner  join  app_employee engineer on detail.employee_id=engineer.id
#  where detail.incident_master_id = ?????????????

#  """


In [ ]:
_ILLEGAL_CHARACTERS_RE = re.compile(r"[\000-\010]|[\013-\014]|[\016-\037]")
# \r\n\r
# https://www.geeksforgeeks.org/python-removing-newline-character-from-string/
def replace_ILLEGAL_CHARACTERS(text):
   text_fixed = _ILLEGAL_CHARACTERS_RE.sub("", text)
   text_fixed=  text_fixed.replace("\n", " ").replace("\r", " ")
   return  text_fixed  

print("Create all issues dataframe")

# dict_params={"company_id_param":company_id_query,"start_date_param":start_date_query,"end_date_param":end_date_query}
# dict_params={"company_id_param":company_id_query,"start_date_param":start_date_query}

df_all=list_data(sql_incident,None,get_postgres_conn())
# df_all['imported_at']=dt_imported
# df_all['imported_at']=df_all['imported_at'].dt.strftime('%Y-%m-%d %H:%M:%S')

df_all['subject']=df_all['subject'].apply(replace_ILLEGAL_CHARACTERS)   
df_all['description']=df_all['description'].apply(replace_ILLEGAL_CHARACTERS) 

                     
# dateNone_cols=['close_datetime']
# for index,row in df_all.iterrows():
#   for dateNone in dateNone_cols:
#     if  row[dateNone] is None:
#         print(row['incident_no']+ " is updated on None in "+dateNone)
#         df_all.loc[index, dateNone] =row['updated_at']

In [ ]:
print(df_all.info())
df_all.tail()


In [ ]:
# icident / file
# for index, serie in df_all.iterrows(): 
#     id=serie["id"]
#     serie = serie.drop('id')
#     # print(serie )
#     # print("==================================================================")
#     with open(f'id_incident\\{id}-incident.ndjson', 'w',encoding='utf8') as f:
#         json_object = json.dumps(serie.to_dict(),ensure_ascii=False)
#         f.write(json_object + '\n')


#  1 files contaim all incidents
with open(f'incident.ndjson', 'w',encoding='utf8') as f:
    for index, serie in df_all.iterrows(): 
        id=serie["id"]
        serie = serie.drop('id')
        json_object = json.dumps(serie.to_dict(),ensure_ascii=False)
        f.write(json_object + '\n')

In [ ]:
#df_all.to_excel("Incident.xlsx",index=False)

    
    # incident_update_at= incident['imported_at']
#     df_detail = list_data(sql_detail,{"incident_id_param": id},get_postgres_conn()) 
#     if df_detail.empty==False:
    
#       for index,row in df_detail.iterrows():
#         if  row['task_end'] is None:
#            print(f"DetailId-{row['id']} of IncidentID-{row['incident_id']}  is updated on None in task_end") 
#            df_detail.loc[index, 'task_end'] =  incident_update_at #row['updated_at']
           
#       df_detail=df_detail.drop(columns=['incident_id','id'])  
#       json_detail = json.loads(df_detail.to_json(orient = 'records'))
#       # print(json_detail)
#       incident['incident_detail']=json_detail
        
    # else:
    #  print(f'no detail in incident {id}')
    #  #incident['incident_detail']=None
    
# no_incident=len(json_incident)
# print(f"No incident: {no_incident}")
# if len(json_incident)>0:
#     print(json_incident[0])

In [ ]:

# job_config = bigquery.LoadJobConfig(
#       schema = [              
#     bigquery.SchemaField("company", "STRING", mode="REQUIRED"),
#     bigquery.SchemaField("incident_no", "STRING", mode="REQUIRED"),
#     bigquery.SchemaField("productype_name", "STRING", mode="REQUIRED"),
#     bigquery.SchemaField("brand_name", "STRING", mode="REQUIRED"),
#     bigquery.SchemaField("model_name", "STRING", mode="REQUIRED"),
#     bigquery.SchemaField("serial_number", "STRING", mode="REQUIRED"), 
#     bigquery.SchemaField("datacenter_name", "STRING", mode="REQUIRED"), 
#     bigquery.SchemaField("branch_name", "STRING", mode="REQUIRED"),       
#     bigquery.SchemaField("severity_id", "INTEGER", mode="REQUIRED"),
#     bigquery.SchemaField("severity_name", "STRING", mode="REQUIRED"),
#     bigquery.SchemaField("status_id", "INTEGER", mode="REQUIRED"),
#     bigquery.SchemaField("status_name", "STRING", mode="REQUIRED"),
#     bigquery.SchemaField("service_type_id", "INTEGER", mode="REQUIRED"),
#     bigquery.SchemaField("service_type_name", "STRING", mode="REQUIRED"),
#     bigquery.SchemaField("incident_type_id", "INTEGER", mode="REQUIRED"),
#     bigquery.SchemaField("incident_type_name", "STRING", mode="REQUIRED"),
#     bigquery.SchemaField("failure_type", "STRING", mode="NULLABLE"),          
#     bigquery.SchemaField("open_datetime", "DATETIME", mode="REQUIRED"),
#     bigquery.SchemaField("close_datetime", "DATETIME", mode="REQUIRED"),
#     bigquery.SchemaField("response_datetime", "DATETIME", mode="REQUIRED"),
#     bigquery.SchemaField("resolved_datetime", "DATETIME", mode="REQUIRED"),
#     bigquery.SchemaField("updated_at", "DATETIME", mode="REQUIRED"),
#     bigquery.SchemaField("install_date", "DATETIME", mode="NULLABLE"),
#     bigquery.SchemaField("eos_date", "DATETIME", mode="NULLABLE"),
#     bigquery.SchemaField("customer_warranty_start", "DATETIME", mode="REQUIRED"),
#     bigquery.SchemaField("customer_warranty_end", "DATETIME", mode="REQUIRED"),
#     bigquery.SchemaField("imported_at", "DATETIME", mode="REQUIRED"),      
#     bigquery.SchemaField(
#     "incident_detail",
#     "RECORD",
#     mode="REPEATED",
#     fields=[
#         bigquery.SchemaField("task_start", "DATETIME", mode="REQUIRED"),
#         bigquery.SchemaField("task_end", "DATETIME", mode="REQUIRED"),
#         bigquery.SchemaField("service_team_name", "STRING", mode="REQUIRED"),
#         bigquery.SchemaField("engineer_name", "STRING", mode="REQUIRED"),
#         bigquery.SchemaField("updated_at", "DATETIME", mode="REQUIRED"),
#     ],),    
#     ]
   
# )
# # for meta in job_config.schema:
# #  print(meta)

# job_config.source_format = bigquery.SourceFormat.NEWLINE_DELIMITED_JSON
# #job_config.schema_update_options = [bigquery.SchemaUpdateOption.ALLOW_FIELD_ADDITION]
# job_config.write_disposition = bigquery.WriteDisposition.WRITE_APPEND  
# #job_config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE


# job = client.load_table_from_json(json_incident,table_id, job_config = job_config)
# if job.errors is not None:
#     print(job.error_result)
#     print(job.errors)
# else:
#     print(f"import to bigquery successfully  {no_incident} records")